# 1. Загрузка и подготовка данных

Отзывы для обучения уже загружены скриптом в SQLite3 бд.

Часть данных была вручную размечена для улучшения качества. Эти датасеты не включены в репозиторий, однако я готов ими поделиться.

In [6]:
import pandas as pd
import numpy as np
import sqlite3
import html
%matplotlib inline

In [9]:
train_data = pd.read_csv("./train_data.csv")
train_data["label"] = train_data["label"].map({"pos":1,"neg":0})
train_data.head()

,Unnamed: 0,text,label
0,0,"Плюсы: Хорошая камера, получаются четкие снимк...",1
1,1,"Плюсы: Это мой четвертый Xiaomi, один лучше др...",1
2,2,"Плюсы: безрамочный, цвета оч. сочные, камера 6...",1
3,3,"Плюсы: Мощный процессор, 6 Gb памяти, отличная...",1
4,4,"Плюсы: Яркий экран, отличное качество фото. Ми...",1


In [14]:
data = train_data

## 1.1 Подготовка текста

Я буду использовать два варианта: довольно стерильные наборы слов и +- оригинальные данные.

In [15]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import _VectorizerMixin

In [16]:
import re

class RoughPreprocessor(TransformerMixin):
    def __init__(self):
        pass
    
    def fit_transform(self, data, y=None):
        return list(map(self.normalize_text_re, map(self.normalize_text, data)))
    
    def normalize_text(self, text):
        _t = html.unescape(text)
        _t = _t.replace("Плюсы: ",". ")
        _t = _t.replace("Минусы: ",". ")
        _t = _t.replace("Впечатления: ",". ")
        _t = _t.replace("<p>"," ")
        _t = _t.replace("</p>", " ")
        _t = _t.replace("\n", " ")
        _t = _t.replace("\r", " ")
        _t = _t.replace("\t", " ")
        _t = _t.replace('"', " ")
        _t = _t.lower()
        return _t.strip()
    
    def normalize_text_re(self, text):
        _t = text
        _t = re.sub(r"[\s.,\-\+><;:!?()]", " ", _t)
        _t = re.sub(r"\s+", " ", _t)
        return _t.strip()
    
    def fit(self, data, y=None):
        return self.fit_transform(data)
    
    def transform(self, data, y=None):
        return self.fit_transform(data) 

In [17]:
# dummy препроцессор, чтоб проще пайплайн строить

In [18]:
class DummyPreprocessor(TransformerMixin):
    def __init__(self):
        pass
    
    def fit_transform(self, data, y=None):
        return data
    
    def transform(self, data, y=None):
        return data
    
    def fit(self, data, y=None):
        return data

## 2. Векторизация

Я хочу воспользоваться библиотекой gensim, потому что я уже использовал их эмбеддинги, и они отлично показали себя в классификации, даже на плохо обработанном датасете.

Чтобы потом удобно запаковать это в sklearn pipeline, я реализую свой класс по образу `TfIdfVectorizer` из `sklearn.feature_extraction`

Также я попробую TfIdfVectorizer и CountVectorizer сам по себе для сравнения какой лучше

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [20]:
import gensim
from gensim.utils import simple_preprocess
from gensim.models import doc2vec
from tqdm import tqdm

import numpy as np
import re
import html

class GensimVectorizer(_VectorizerMixin, BaseEstimator):
    def __init__(self):
        pass

    def fit(self, raw_documents, y=None):
        X = self.preprocess(raw_documents)
        # print("Creating model...")
        model = gensim.models.doc2vec.Doc2Vec(
            vector_size=100, 
            min_count=10,
            epochs=40
        )
        # print("Building vocab...")
        model.build_vocab(X)
        # print("Training doc2vec...")
        model.train(X, total_examples=model.corpus_count, epochs=model.epochs)
        self.model = model
        return self

    def transform(self, raw_documents, y=None):
        X = self.preprocess(raw_documents)
        # print("Iinferring vectors...")
        vectorized_texts = []
        for doc_id, _ in enumerate(tqdm(X, desc="Inferring vectors: ")):
            inferred_vector = self.model.infer_vector(X[doc_id].words)
            vectorized_texts.append(inferred_vector)

        return vectorized_texts

    def preprocess(self, raw_documents):
        # print("Tokenization...")
        processed_texts = []
        for idx, text in enumerate(tqdm(raw_documents, desc="Tokenization: ")):
            processed_texts.append(doc2vec.TaggedDocument(simple_preprocess(text), [idx]))
        return processed_texts


    def fit_transform(self, texts, y=None) -> np.ndarray:
        self.fit(texts)
        X = self.transform(texts)
        
        return np.array(X)

## 3. Обучение модели 

Модели, которые я буду рассматривать:
+ LogisticRegression
+ GradientBoosting
+ LinearSVC

In [21]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [22]:
X = data["text"]
# y = data.label.map({"pos": 1, "neg": 0})
y = data.label

In [74]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate

_model = make_pipeline(DummyPreprocessor(), CountVectorizer(ngram_range=(1,2)), LinearSVC(max_iter=4000, class_weight="balanced"))

In [75]:
_model.fit(X, y)

Pipeline(steps=[('dummypreprocessor',
                 <__main__.DummyPreprocessor object at 0x7f391c33f790>),
                ('countvectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('linearsvc',
                 LinearSVC(class_weight='balanced', max_iter=4000))])

In [76]:
cross_val_score(_model, X, y, n_jobs=6, cv=4)

array([0.88442211, 0.85678392, 0.85678392, 0.8790932 ])

## 4. Улучшение модели

Улучшить модель можно несколькими путями:
+ 1. Попробовать разный препроцессинг текста
+ 2. Подобрать параметры модели векторизации
+ 3. Подобрать параметры классификатора
+ 4. Попробовать другие классификаторы
+ 5. Сбалансировать классы для обучения
+ 0. Делать перебор не по сетке, а более "разумными" методами

Я сделаю только пп. 1, 3

In [24]:
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.pipeline import Pipeline

pipeline_ = Pipeline([
    ("preprocessor", RoughPreprocessor()),
    ("vec", GensimVectorizer()),
    ("clf", GradientBoostingClassifier())
])
# я разбил на несколько словарей потому что 
# параметры не унифицированы между классами моделей
linear_classifiers = {
    "clf":[LogisticRegression(), LinearSVC()],
    "clf__class_weight":["balanced"],
    "clf__dual": [True, False]
}
forest_classifiers = {
    "clf":[GradientBoostingClassifier(), RandomForestClassifier()],
    "clf__n_estimators":[100,200, 500],
    "clf__max_depth": [3,4,5,10,20],
}
preprocessors = {
    "preprocessor":[RoughPreprocessor(), DummyPreprocessor()],
}
common_vectorizers = {
    "vec": [CountVectorizer(), TfidfVectorizer()],
    "vec__ngram_range": [(1,1), (1,2), (1,3), (1,4), (1,5)],
    "vec__min_df": [1,2,3,4,10],
    "vec__max_features": [None, 200, 500, 1000]
}
embedding_vectorizers = {
    "vec": [GensimVectorizer()]
}

param_grid = [
{
    # Plain Vectorizers + Linear Models
    **preprocessors,
    **common_vectorizers,
    **linear_classifiers    
}, 
{
    # Gensim Embeddings + Linear Models
    **preprocessors,
    **embedding_vectorizers,
    **linear_classifiers,
}, 
{
    # Gensim Embeddings + GradBoost Models
    **preprocessors,
    **embedding_vectorizers,
    **forest_classifiers,
}, 
{
    # Plain Vectorizers + GradBoost Models
    **preprocessors,
    **common_vectorizers,
    **forest_classifiers,
}
]
    

In [ ]:
%%time
# ~ несколько недель на 6 ядрах Ryzen3
gscv_ = GridSearchCV(
    estimator=pipeline_,
    param_grid=param_grid, 
    scoring="accuracy", 
    n_jobs=6, refit=True, cv=3, verbose=2)
gscv_.fit(X, y)

In [65]:
print(gscv_.best_score_)
print(gscv_.best_params_)

0.8680074382025134
{'clf': LinearSVC(class_weight='balanced'), 'clf__class_weight': 'balanced', 'clf__dual': True, 'preprocessor': <__main__.DummyPreprocessor object at 0x7f82589ecd60>, 'vec': CountVectorizer(min_df=2, ngram_range=(1, 4)), 'vec__min_df': 2, 'vec__ngram_range': (1, 4)}


Лучший набор параметров:


In [27]:
model_final = gscv_.best_estimator_

In [77]:
model_final = _model

## 5. Инференс и подготовка сабмита

In [36]:
from sklearn.metrics import accuracy_score

In [24]:
# это для импорта предоставленного файла
import bs4
test = []
with open("test.csv") as tfile:
    sp = bs4.BeautifulSoup(tfile)
    revs = sp.findAll("review")
    for r in revs:
        test.append(r.text)

# pd.read_csv("test.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'test.csv'

In [29]:
# а это мой размеченный тестовый файл
test = pd.read_csv("./test_.csv")

In [52]:
test["prediction"] = model_final.predict(test.text)

In [54]:
# моя разметка тестового файла не очень точная.
accuracy_score(test["y"], test["prediction"])

0.8

In [55]:
submission = test.copy()
submission["y"] = test.prediction.map({1: "pos", 0: "neg"})
submission.to_csv("./submission.csv", columns=["y"], index_label="Id")

## 6. Упаковка модели

In [40]:
import pickle as pkl
import dill

In [78]:
# В финальный пайп я пакую предобученную на большой выборке модель векторизации 
# и классификатор, обученный на выборке после ресемплинга
final_pipeline = model_final

In [79]:
with open("../SentimentModelRU.pkl", "wb") as fout:
    dill.dump(final_pipeline, fout)

In [58]:
with open("../SentimentModelRU.pkl", "rb") as fin:
    v = dill.load(fin)